In [11]:
# !pip install snowflake-connector-python
# !pip install load_dotenv
# !pip install python-dotenv
!pip install snowflake-connector-python snowflake-sqlalchemy pandas

In [9]:

!pip install SQLAlchemy



In [ ]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import snowflake.connector
import os
load_dotenv(override=True)


In [ ]:
def load_data_from_env():
  load_dotenv(override=True)
  account = os.getenv("SNOWFLAKE_ACCOUNT")
  user = os.getenv("SNOWFLAKE_USER")
  password = os.getenv("SNOWFLAKE_PASSWORD")
  warehouse = os.getenv("SNOWFLAKE_warehouse")
  database = os.getenv("SNOWFLAKE_database")
  schema = os.getenv("snowflake_schema")
  return user,account, password, warehouse,database

user, account, password, warehouse, database = load_data_from_env()


print(user, account, password, warehouse, database)

In [ ]:
def connection_snowflake_database():
  conn = snowflake.connector.connect(
        user=user,
        password=paasword,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
  conn = conn.cursor()
  conn.execute('select  current_database()').fetchone()
  print('database is connected ')
  return conn

In [ ]:
def execute_multi_query(conn):
    """Executes multiple SQL queries."""
    try:
        queries = [
            "use database ubio_demo;",
            "use schema ubio_schema;"
        ]
        for query in queries:
            conn.execute(query)

        print("Queries executed successfully.")
    except Exception as e:
        print(f"Error executing queries: {e}")
     # Ensure connection is closed in case of errors

# Assuming 'connection_snowflake_database' function returns an active connection
conn = connection_snowflake_database()
execute_multi_query(conn)

In [ ]:
# test database  connection 
conn.execute("select current_database(), current_schema()").fetchone()

In [ ]:
def create_database_and_schema(conn):
  """createa new database and schema where will can load the data"
  """
  try:
    query=[
        "create database if not exists uk_demo;",
        "use database ubio_demo;",
        "create schema if not exists uk_schema;",
        "use schema ubio_schema;"


    ]
    for query in query:
      conn.execute(query)
    print("database and schema created successfully")
  except Exception as e:
    print(f"Error executing queries: {e}")
conn=connection_snowflake_database()
create_database_and_schema(conn)

In [ ]:
def create_file_format_stage(conn):
    """
    Create a file format and stage in Snowflake, and upload a file to the stage.
    """
    try:
        # Define the queries to create file format, stage, and upload the file
        file_path ="/content/railway.csv"
        with open(file_path, 'rb') as file:
          file_path = file.read()

        queries = [
            """
            CREATE OR REPLACE FILE FORMAT my_csv_file_format
            TYPE = 'CSV'
            FIELD_DELIMITER = ','
            SKIP_HEADER = 2;
            """,
            """
            CREATE OR REPLACE STAGE my_csv_stage
            FILE_FORMAT = my_csv_file_format;
            """,
            """
            PUT file:///content/railway.csv @my_csv_stage;
            """
        ]

        # Execute each query in the list

        for query in queries:
            conn.execute(query)
            print(f"Executed: {query.strip().splitlines()[0]}...")

        print("File format and stage created successfully.")
        print("File loaded into the stage.")

    except Exception as e:
        print(f"Error executing queries: {e}")


conn = connection_snowflake_database()
create_file_format_stage(conn)


In [ ]:
import pandas as pd

def check_if_the_file_is_loaded(conn):
    """
    Check the Snowflake stage to verify if the file is loaded successfully.
    """
    try:
        # Query to list files in the stage
        query = "LIST @my_csv_stage;"

        conn.execute(query)
        result = conn.fetchall()

        columns = [desc[0] for desc in conn.description]

        df = pd.DataFrame(result, columns=columns)
        print("Stage Contents:")
        print(df)
        print("File is loaded successfully into the stage.")

    except Exception as e:
        print(f"Error executing query: {e}")

conn = connection_snowflake_database()
check_if_the_file_is_loaded(conn)



In [ ]:
import pandas as pd

def load_data_to_table(conn):
    """
    Load data from the Snowflake stage into a table.
    """
    try:
        # List of queries to execute
        queries = [
            """
            CREATE OR REPLACE TABLE uk_train_table AS
            SELECT
                CAST($1 AS STRING) AS Transaction_ID,
                CAST($2 AS DATE) AS Date_of_Purchase,
                CAST($3 AS TIME) AS Time_of_Purchase,
                CAST($4 AS VARCHAR) AS Purchase_Type,
                CAST($5 AS VARCHAR) AS Payment_Method,
                CAST($6 AS VARCHAR) AS Railcard,
                CAST($7 AS VARCHAR) AS Ticket_Class,
                CAST($8 AS VARCHAR) AS Ticket_Type,
                CAST($9 AS DECIMAL(10,2)) AS Price,
                CAST($10 AS VARCHAR) AS Departure_Station,
                CAST($11 AS VARCHAR) AS Arrival_Destination,
                CAST($12 AS DATE) AS Date_of_Journey,
                CAST($13 AS TIME) AS Departure_Time,
                CAST($14 AS TIME) AS Arrival_Time,
                CAST($15 AS TIME) AS Actual_Arrival_Time,
                CAST($16 AS VARCHAR) AS Journey_Status,
                CAST($17 AS VARCHAR) AS Reason_for_Delay,
                CAST($18 AS VARCHAR) AS Refund_Request
            FROM @my_csv_stage
            """,
            """
            SELECT * FROM uk_train_table;
            """
        ]

        # Execute each query
        for query in queries:

            conn.execute(query)


            if query.strip().lower().startswith("select"):
                result = conn.fetchall()
                columns = [desc[0] for desc in conn.description]
                df = pd.DataFrame(result, columns=columns)
                print("Data loaded into DataFrame:")
                print(df.head())



    except Exception as e:
        print(f"Error executing query: {e}")
    return df

conn = connection_snowflake_database()
df =load_data_to_table(conn)


In [ ]:
def transformation_of_data(df):
    """
    Perform basic data transformations and return insights about the DataFrame.
    Args:
        df (pd.DataFrame): The input DataFrame.
    Returns:
        dict: A dictionary containing column names, null counts, and data types.
    """
    try:
        result = {
            "columns": df.columns.tolist(),
            "null_counts": df.isnull().sum().to_dict(),
            "data_types": df.dtypes.to_dict(),
            'change_data_type' : df.PRICE.astype('int').to_dict()}
        return result
    except Exception as e:
        print(f"Error during transformation: {e}")
        return None


transformation_of_data(df)


In [ ]:
def create_deminsional_table(df):
  """"create the deminsional table from the dataset"""
  Dim_railcard= df['RAILCARD'].rename({'RAILCARD':'railcard'})
  Dim_railcard.index.name ='railcard_id'
  Dim_railcard=Dim_railcard.reset_index()
  Dim_railcard.to_sql('Dim_railcard', con=engine, if_exists='replace', index=False)
  return Dim_railcard

create_deminsional_table(df)


In [ ]:
#df.to_sql('uk_train_table', con=engine, if_exists='replace', index=False)
df.index.name = 'Ticket_id'
df = df.reset_index()
df.to_sql('uk_train_table', con=engine, if_exists='replace', index=False)
df.to_csv('fact_table', index=False)
DF=pd.read_csv('fact_table')
DF.head()

In [ ]:
Dim_railcard= df['RAILCARD'].rename({'RAILCARD':'railcard'})
Dim_railcard.index.name ='railcard_id'
Dim_railcard=Dim_railcard.reset_index()
Dim_railcard.to_sql('Dim_railcard', con=engine, if_exists='replace', index=False)

In [ ]:
def create_table_tciket_id(df):
  """create a table for ticket id
  """
  Dim_Tickets=df[['TICKET_CLASS', 'TICKET_TYPE']]
  Dim_Tickets.index.name ='Ticket_id'
  Dim_Tickets=Dim_Tickets.reset_index()
  Dim_Tickets.to_sql('Dim_Tickets', con=engine, if_exists='replace', index=False)
  return Dim_Tickets
create_table_tciket_id(df)

In [ ]:
Dim_Tickets=df[['TICKET_CLASS', 'TICKET_TYPE']]
Dim_Tickets.index.name ='Ticket_id'
Dim_Tickets=Dim_Tickets.reset_index()
Dim_Tickets.to_csv('Dim_Tickets.csv',index=False)


In [ ]:
"""create the Dim_station table from the dataset
"""
Dim_Station = df[["DEPARTURE_STATION" ,"ARRIVAL_DESTINATION"]]
Dim_Station.index.name ='Station_id'
Dim_Station=Dim_Station.reset_index()
Dim_Station=Dim_Station.rename(columns={'DEPARTURE_STATION':'Departure_Station','ARRIVAL_DESTINATION':'Arrival_Destination'})
Dim_Station.to_sql('Dim_Station', con=engine, if_exists='replace', index=False)

Dim_Station.to_csv('Dim_Station.csv',index=False)


In [ ]:
"""Create the Refund_ID table from the dataset"""
Refund_ID = df[["DATE_OF_JOURNEY", "REFUND_REQUEST"]]
Refund_ID.index.name = 'Refund_ID'
Refund_ID = Refund_ID.reset_index()
Refund_ID = Refund_ID.rename(columns={'DATE_OF_JOURNEY': 'Date_of_Journey', 'REFUND_REQUEST': 'Refund_Request'})
Refund_ID.to_sql('Refund_ID', con=engine, if_exists='replace', index=False)
Refund_ID.to_csv('Dim_Journeys.csv', index=False)


In [ ]:
"""create the Dim_Journeys table from the Dim_refund table"""
Dim_Journeys=df[["REASON_FOR_DELAY" ,"DEPARTURE_TIME","ARRIVAL_TIME","ACTUAL_ARRIVAL_TIME" ,"JOURNEY_STATUS"]]
Dim_Journeys.index.name ='Journey_id'
Dim_Journeys=Dim_Journeys.reset_index()
Dim_Journeys=Dim_Journeys.rename(columns={'DATE_OF_JOURNEY':'Date_of_Journey' ,'DEPARTURE_TIME':'Departure_Time','ARRIVAL_TIME':'Arrival_Time','ACTUAL_ARRIVAL_TIME':'Actual_Arrival_Time'})
Dim_Journeys.to_sql('Dim_Journeys', con=engine, if_exists='replace', index=False)
Dim_Journeys.to_csv('Dim_Journeys.csv',index=False)

In [ ]:
import pandas as pd
import datetime as detetime
import datetime as dt
"""" create the Dim_date table"""

df["Date_of_Purchase"] = pd.to_datetime(df["DATE_OF_PURCHASE"])
df["day_name"] = df["Date_of_Purchase"].dt.day_name()
df["month_name"] = df["Date_of_Purchase"].dt.month_name()
df["Year"] = df["Date_of_Purchase"].dt.year
df["Quarter"] = df["Date_of_Purchase"].dt.quarter
# Convert 'TIME_OF_PURCHASE' to strings before applying pd.to_datetime
df["Hour_of_Purchase"] = pd.to_datetime(df["TIME_OF_PURCHASE"].astype(str)).dt.hour
Dim_date = df[["Date_of_Purchase", "day_name", "month_name", "Year", "Quarter", "Hour_of_Purchase"]]
Dim_date.to_sql('Dim_date', con=engine, if_exists='replace', index=False)
Dim_date.index.name = 'Date_id'
Dim_date = Dim_date.reset_index()
Dim_date.to_sql('Dim_date', con=engine, if_exists='replace', index=False)
Dim_date.to_csv('Dim_date.csv', index=False)

In [ ]:
import pandas as pd

df["Hour_of_Purchase"] = df["TIME_OF_PURCHASE"].apply(lambda x: x.hour)
df["Minute_of_Purchase"] = df["TIME_OF_PURCHASE"].apply(lambda x: x.minute)
df["Second_of_Purchase"] = df["TIME_OF_PURCHASE"].apply(lambda x: x.second)
Dim_time = df[["TIME_OF_PURCHASE", "Hour_of_Purchase", "Minute_of_Purchase", "Second_of_Purchase"]]
Dim_time.index.name = 'Time_id'
Dim_time = Dim_time.reset_index()
Dim_time.to_sql('Dim_time', con=engine, if_exists='replace', index=False)
Dim_time.to_csv('Dim_time.csv', index=False)
print(Dim_time)

In [ ]:
import pandas as pd
df["Date_of_Journey"] = pd.to_datetime(df["DATE_OF_JOURNEY"])
df["day_name_journey"] = df["Date_of_Journey"].dt.day_name()
df["month_name_journey"] = df["Date_of_Journey"].dt.month_name()
df["Year_journey"] = df["Date_of_Journey"].dt.year
df["Quarter"] = df["Date_of_Journey"].dt.quarter
Date_ID_Departure = df[["Date_of_Journey", "day_name_journey", "month_name_journey", "Year_journey", "Quarter"]]
Date_ID_Departure.index.name = 'Date_ID_Departure'
Date_ID_Departure = Date_ID_Departure.reset_index()
Date_ID_Departure.to_sql('Date_ID_Departure', con=engine, if_exists='replace', index=False)
Date_ID_Departure.to_csv('Date_ID_Departure.csv', index=False)
print(Date_ID_Departure)




